### Data file for Return Model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

df = pd.read_csv("mpd_sp500.csv")

In [2]:
# drop columns that has "maturity_target" , "lg_change_decr", and "lg_change_incr" in the column name; those are irrelevant for feature selection
df = df[df.columns.drop(list(df.filter(regex='maturity_target')))]
df = df[df.columns.drop(list(df.filter(regex='lg_change_decr')))]
df = df[df.columns.drop(list(df.filter(regex='lg_change_incr')))]
# drop column of SP_adj_close and SP_volume
df = df.drop(columns=['SP_adj_close', 'SP_volume', 'SP_lg_ret', 'SP_lg_ret_vol'])

###  Box Jenkins Methodology
- Based on the ACF and PACF plots of SPX log return, we can construct AR and MA models with in 6 lags by using cross validation
- 通常情况下，在Box Jenkins建模的时候，虽然我们管它叫ARIMA建模，但是我们建模的时候只考虑AR和I的部分。不考虑MA的部分, 因为这样整体的建模可以不依赖于ARIMA的MLE的框架
- Then we can use the Lasso and Ridge regression to select the best model
- Finally, we can use the ARMA model to forecast the future log return
- 用Log Price来建立ARIMA模型，LogPrice arima(3,1,0) = LogReturn arima (3,0,0), 因为用log price来对你RMSE的话会显得模型更准一点
- AIC BIC 来判断模型对于in sample的拟合效果

### construct lagged variables from f1_mu to f14_prInc from lag 1 to lag 6

In [3]:
# Generate lagged variables from f1_mu to SP_lg_pr
lags = 6
for lag in range(1, lags+1):
    # for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('SP_lg_ret_vol')+1]:
    for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('SP_lg_pr')+1]:    
        df[f'{col}_lag{lag}'] = df[col].shift(lag)

/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_88853/2923299393.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_88853/2923299393.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_88853/2923299393.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many

In [4]:
# to csv file
df.to_csv('mpd_sp500_lagged.csv', index=False)

### Data file for Volatility Model

In [29]:
df_vol = pd.read_csv("mpd_sp500.csv")

# Drop irrelevant columns
df_vol = df_vol[df_vol.columns.drop(list(df_vol.filter(regex='maturity_target')))]
df_vol = df_vol[df_vol.columns.drop(list(df_vol.filter(regex='lg_change_decr')))]
df_vol = df_vol[df_vol.columns.drop(list(df_vol.filter(regex='lg_change_incr')))]
df_vol = df_vol.drop(columns=['SP_adj_close', 'SP_volume', 'SP_lg_pr'])

# Generate lagged variables from f1_mu to SP_lg_pr
lags = 6
for lag in range(1, lags+1):
    for col in df_vol.columns[df_vol.columns.get_loc('f1_mu'):df_vol.columns.get_loc('SP_lg_ret_vol')+1]:    
        df_vol[f'{col}_lag{lag}'] = df_vol[col].shift(lag)

/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_88853/1356736937.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_vol[f'{col}_lag{lag}'] = df_vol[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_88853/1356736937.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_vol[f'{col}_lag{lag}'] = df_vol[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_88853/1356736937.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling 

In [30]:
df_vol.to_csv('mpd_sp500_lagged_vol.csv', index=False)

AR(1) has the lowest AIC and BIC score, so choose AR(1)

##### Modelling the Price with Lasso

Lasso Model : $ RSS +  \lambda + penalty$

- Choose Lambda ($\lambda$) by
    1. cross validation
    2. adaptive
    3. plugin 


#### Lasso (cross validation)

在这里要的ARIMA模型中，我们只考虑AR和I的部分，不考虑MA的部分， 因为我们要对比lasso模型，lasso模型不考虑error部分的serial correlation，意味着在使用lasso进行时间序列分析时，模型不会特别处理或识别误差项(error term)中的序列相关性(serial correlation)。序列相关性是指在时间序列数据中，前一个时刻的误差项可能会影响到下一个时刻的误差项。
在时间序列分析中，ARMA（自回归移动平均）模型是一种常用的方法，它由两部分组成：AR（自回归）部分和MA（移动平均）部分。AR部分考虑了当前值与其过去值之间的关系，而MA部分则考虑了当前误差项与其过去误差项之间的关系。
当提到lasso模型不考虑error部分的serial correlation，特别指的是在lasso模型中，MA（移动平均）部分不会被特别考虑或识别。这是因为lasso（最小绝对收缩和选择算子）是一种用于回归分析的方法，它通过对系数的大小施加惩罚来实现变量的选择和复杂度的控制，从而帮助避免过拟合并提高模型的预测能力。然而，lasso主要关注于变量选择和正则化，并不直接处理误差项的序列相关性问题。
因此，当使用lasso方法分析时间序列数据时，如果数据中存在显著的误差序列相关性，尤其是MA部分的影响，这种序列相关性可能不会被lasso模型适当地捕捉或调整。这就需要研究者在使用lasso方法前，对时间序列数据进行适当的预处理，或者在模型评估和解释时考虑到这个限制。